In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import numpy as np
import re
import string

In [ ]:
institutional_type = pd.read_csv('../../data/institutions_us.csv')
institutions_impact = pd.read_excel('/data/datasets/WOS/US/institutions_impact.xlsx', names = ['Institution','country', 'npapers','avg_citations'])

In [ ]:
address = pd.read_pickle('/data/datasets/WOS/US/Adresses_impact.p')

In [ ]:
grid_df = pd.read_csv('../../data/grid-2021-09-16/grid.csv')
grid_type = pd.read_csv('../../data/grid-2021-09-16/full_tables/types.csv')

In [ ]:
## updated to new 2021 classification
# carnegie_df = pd.read_csv('../../data/carnegie/Data_1-7-2022---920.csv')
# carnegie_labels = pd.read_csv('../../data/carnegie/ValueLabels_1-7-2022---920.csv')
# carnegie_df = carnegie_df[['UnitID','Institution Name']]
carnegie_df = pd.read_excel('../../data/carnegie/CCIHE2021-PublicData.xlsx',sheet_name='Data')
carnegie_df = carnegie_df[['unitid','name']]
carnegie_df.columns = ['carnegie_id','carnegie_name']

carnegie_df['carnegie_id'] = carnegie_df.carnegie_id.astype("category")

In [6]:
institution_count = address.Institution.value_counts().reset_index()
institution_count.columns=['name','n']

In [7]:
institution_count['name'] = institution_count.name.str.upper()

In [8]:
institution_count = institution_count.merge(institutional_type, left_on='name', right_on='Institution', how='left')

institution_count.drop(columns='Institution',inplace=True)

In [9]:
institution_count.sector.value_counts()

hospital      64251
company       51456
university    23784
goverment      3349
Name: sector, dtype: int64

## Automatic cleaning

In [10]:
def rename_to_greater(institution_count,to_omit, threshold):
    
    ic = institution_count.copy()
    names = to_omit

    ic['new_name'] = ic.name

    for index, row in tqdm(ic[ic.n>threshold].iterrows()): # I only clean names of institutions with n>100, 5K institutions
        if row.new_name in names:
            continue
        else:
            name = row['new_name']
            sector = row['sector']
            ic_names = ic.new_name.values
            matched_names = ic_names[[bool(re.search(pattern=f'(^|\s|-){name}($|\s|-)', string=n)) for n in ic_names]]
            ic.loc[ic.name.isin(matched_names),'new_name'] = name
            ic.loc[ic.name.isin(matched_names),'sector'] = sector
            names += [name]

    return ic

In [11]:
states = ['ALASKA', 'ALABAMA', 'ARKANSAS', 'AMERICAN SAMOA', 'ARIZONA', 'CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'COLUMBIA', 'DELAWARE', 'FLORIDA', 'GEORGIA', 'GUAM', 'HAWAII', 'IOWA', 'IDAHO', 'ILLINOIS', 'INDIANA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MASSACHUSETTS', 'MARYLAND', 'MAINE', 'MICHIGAN', 'MINNESOTA', 'MISSOURI', 'MISSISSIPPI', 'MONTANA', 'NORTH CAROLINA', 'NORTH DAKOTA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NEVADA', 'NEW YORK', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA','PENN', 'PUERTO RICO', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VIRGINIA', 'VIRGIN ISLANDS', 'VERMONT', 'WASHINGTON', 'WISCONSIN', 'WEST VIRGINIA', 'WYOMING']

In [12]:
state_unis = ['UNIV-' + state for state in states] + [state + '-UNIV' for state in states]

In [13]:
to_omit = ['LLC','MED-CTR','RES-INST', 'CHILDRENS-HOSP','USA','SCH-MED','VET-ADM','UNIV-CALIF','UNC','UNIV-HOSP'] + state_unis

In [ ]:
ic = rename_to_greater(institution_count, to_omit,1000)

0it [00:00, ?it/s]

In [ ]:
ic.new_name.value_counts()[:40]

In [ ]:
new_ic= ic.groupby('new_name').n.sum().reset_index().sort_values('n',ascending=False)

In [ ]:
print(f"we retrieve {round(new_ic[new_ic.n>=500].n.sum()/new_ic.n.sum()*100,2)}% of the database,")
print(f"represented by {len(new_ic[new_ic.n>=500])} institutions,")
print(f"which are the {round(len(new_ic[new_ic.n>=500])/len(new_ic)*100,3)}% of the institutions.")



In [ ]:
large_institutions = new_ic[new_ic.n>=500]

In [ ]:
large_institutions_all = ic[ic.new_name.isin(large_institutions.new_name)]

In [29]:
ic.groupby('sector').n.sum()/ic.groupby('sector').n.sum().sum()

sector
company       0.056296
goverment     0.087184
hospital      0.157070
university    0.699450
Name: n, dtype: float64

In [28]:
institutions_to_clean = large_institutions_all[large_institutions_all.sector.isin(['university','hospital'])]

In [29]:
institutions_to_clean.sort_values(['new_name','n'],ascending=[True,False])

,name,n,sector,new_name
821,ABILENE-CHRISTIAN-UNIV,1635,university,ABILENE-CHRISTIAN-UNIV
152872,ABILENE-CHRISTIAN-UNIV-DALLAS,1,university,ABILENE-CHRISTIAN-UNIV
1814,ABINGTON-MEM-HOSP,511,hospital,ABINGTON-MEM-HOSP
803,ADELPHI-UNIV,1693,university,ADELPHI-UNIV
152712,ADELPHI-UNIV-GARDEN-CITY,1,university,ADELPHI-UNIV
...,...,...,...,...
136370,ZUCKER-HILLSIDE-HOSP-NORTH-SHORE-LONG-ISL-JEWISH,2,hospital,ZUCKER-HILLSIDE-HOSP
136498,ZUCKER-HILLSIDE-HOSP-DIV-NORTH-SHORE,2,hospital,ZUCKER-HILLSIDE-HOSP
201139,ZUCKER-HILLSIDE-HOSP-NORTHSHORE-LIJ,1,hospital,ZUCKER-HILLSIDE-HOSP
218270,N-SHORE-LIJ-ZUCKER-HILLSIDE-HOSP,1,hospital,ZUCKER-HILLSIDE-HOSP


In [30]:
sorter = institutions_to_clean.new_name.unique()

institutions_to_clean.loc[:,'new_name'] = institutions_to_clean.new_name.astype("category")

institutions_to_clean.loc[:,'new_name'] = institutions_to_clean.new_name.cat.set_categories(sorter)


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [31]:
institutions_to_clean = institutions_to_clean.sort_values(['new_name','n'],ascending=[True,False])

### Partial match with GRID

In [32]:
grid_us = grid_df[grid_df.Country == 'United States']

In [33]:
grid_us = grid_us.merge(grid_type,left_on='ID', right_on='grid_id')

In [34]:
def standardize_name(name):
    name = name.translate(str.maketrans('','',string.punctuation)) # remove punctuation
    name = re.sub('the|of','',name,flags=re.IGNORECASE)
    name = re.sub(' +', ' ', name) #remove multiple white space
    name = name.upper()
    name = name.replace(' ','-')
    name = name.replace('UNIVERSITY','UNIV')
    name = name.replace('INSTITUTE','INST')
    name = name.replace('DIVISION','DIV')
    name = name.replace('CLINIQUE','CLI')
    name = name.replace('MEDICAL','MED')    
    name = name.replace('CENTER','CTR')  
    name = name.replace('RESEARCH','RES')  
    name = name.replace('RESEARCH','RES')  
    name = name.replace('COLLEGE','COLL')  
    name = name.replace('ACADEMY','ACAD')      
    name = name.replace('CALIFORNIA','CALIF')      
    name = name.replace('PENNSYLVANIA','PENN')
    return name

In [35]:
text = 'University of California, Los Angeles'
standardize_name(text)

'UNIV-CALIF-LOS-ANGELES'

In [36]:
grid_us.loc[:,'standard_name'] = grid_us.Name.apply(standardize_name)

In [37]:
grid_us_merge = grid_us[['grid_id','Name','standard_name']]
grid_us_merge.columns = ['grid_id','grid_name','grid_standard_name']

In [38]:
def double_merge_grid(institutions_to_clean, grid_us_merge):

    partial_match = institutions_to_clean.merge(grid_us_merge, left_on='new_name', right_on='grid_standard_name', how='left')

    partial_match.drop(columns='grid_standard_name',inplace=True)

    not_matched = partial_match[partial_match.grid_id.isnull()]

    not_matched.drop(columns=['grid_id', 'grid_name'],inplace=True)

    second_match = not_matched.merge(grid_us_merge, left_on='name', right_on='grid_standard_name', how='inner')
    second_match.drop(columns='grid_standard_name',inplace=True)

    partial_match.loc[partial_match.name.isin(second_match.name),['grid_id','grid_name']] = second_match[['grid_id','grid_name']].values

    return partial_match

In [39]:
partial_match = double_merge_grid(institutions_to_clean, grid_us_merge)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Partial match with Carnegie

In [40]:
carnegie_df.loc[:,'carnegie_standard_name'] = carnegie_df.carnegie_name.apply(standardize_name)

In [41]:
def double_merge_carnegie(partial_match, carnegie_df):

    partial_match = partial_match.merge(carnegie_df, left_on='new_name', right_on='carnegie_standard_name', how='left')

    partial_match.drop(columns='carnegie_standard_name',inplace=True)

    not_matched = partial_match[partial_match.carnegie_id.isnull()]

    not_matched.drop(columns=['carnegie_id', 'carnegie_name'],inplace=True)

    second_match = not_matched.merge(carnegie_df, left_on='name', right_on='carnegie_standard_name', how='inner')
    second_match.drop(columns='carnegie_standard_name',inplace=True)

    partial_match.loc[partial_match.name.isin(second_match.name),['carnegie_id', 'carnegie_name']] = second_match[['carnegie_id', 'carnegie_name']].values

    return partial_match

In [42]:
matched_institutions = double_merge_carnegie(partial_match, carnegie_df)

In [44]:
matched_institutions

,name,n,sector,new_name,grid_id,grid_name,carnegie_id,carnegie_name
0,HARVARD-UNIV,302415,university,HARVARD-UNIV,grid.38142.3c,Harvard University,166027,Harvard University
1,HARVARD-UNIV-HERBARIA,143,university,HARVARD-UNIV,grid.38142.3c,Harvard University,166027,Harvard University
2,BROAD-INST-MIT-&-HARVARD-UNIV,141,university,HARVARD-UNIV,grid.38142.3c,Harvard University,166027,Harvard University
3,ARNOLD-ARBORETUM-HARVARD-UNIV,68,university,HARVARD-UNIV,grid.38142.3c,Harvard University,166027,Harvard University
4,HARVARD-UNIV-HEBARIA,67,university,HARVARD-UNIV,grid.38142.3c,Harvard University,166027,Harvard University
...,...,...,...,...,...,...,...,...
14630,UT-SOUTHWESTERN-MED-CTR-PARKLAND-MEM-HOSP,2,hospital,UT-SOUTHWESTERN-MED-CTR,NaN,NaN,NaN,NaN
14631,UT-SOUTHWESTERN-MED-CTR-PARKLAND-MEM-HOSP-DALLAS,1,hospital,UT-SOUTHWESTERN-MED-CTR,NaN,NaN,NaN,NaN
14632,UT-SOUTHWESTERN-MED-CTR-DALLAS-TX,1,hospital,UT-SOUTHWESTERN-MED-CTR,NaN,NaN,NaN,NaN
14633,UT-SOUTHWESTERN-MED-CTR-&-CHILDRENS-HEALTH,1,hospital,UT-SOUTHWESTERN-MED-CTR,NaN,NaN,NaN,NaN


In [45]:
matched_institutions.to_excel('../../handcoding/institutions.xlsx')